In [58]:
import openrouteservice as ors
import folium
from shapely import geometry
from shapely.geometry import Polygon, MultiPolygon

client = ors.Client(key='5b3ce3597851110001cf6248971aa72d85c54387b65a72d6e34b294f')

## Ablauf/ Overview

#### Regen
- Isochronen in 15Minuten-Wege-Abständen, an diesen Punkten (= Kreisen) Wetter abfragen. Pro Isochrone müssen mehrere Punkte ausgewählt werden (Anzahl abhängig vom Radius der jeweiligen Isochrone)
- Erstmal als Vereinfachung: keine tatsächlichen Isochronen verwenden, sondern einfach Kreise mit festgelegten Radien (so vermeiden wir eine API-Abfrage). (Aber super fancy).


- Anzahl der Punkte auf die Ränder der jeweiligen Isochrone legen, sodass diese gesamt abgedeckt ist (Umsetzung tbd), dort dann das Wetter abfragen, in abhängigkeit des bereits zurückgelegten Weges

- Ausgabe sind dann x Punkte, mit Regen Ja/Nein, um diese ziehen wir Radius mit festgelegter Größe (vereinfachte Annahme)
#### POI
- diese Punkte + deren zugehöriger Bereich bilden dann die Ausschluss-Areas für die nachfolgende POI-Abfrage.

- Die POI-Abfrage liefert dann nur Orte, wo die Sonne scheint wenn man ankommt.

- User wählt Ziel aus.

#### Routenberechnung
- möglicherweise ist die Route dann länger als vorher angegeben (da Luftlinie für POI verwendet wird). + Dann noch bei Routenwechsel wegen Regen.
- für die Routenberechnung geben wir dann wieder die Regen-Auschluss-Polygone ein.
- fertig.


In [2]:
# user input:

print("How far would you like to travel today? Please input your answer in kilometers.")
  
distance_input = 2 # [km]

How far would you like to travel today? Please input your answer in kilometers.


## Calculate Points of interest within range

In [59]:
homebase = [10.5288243, 52.2733412] # Universitaet Braunschweig

m = folium.Map(location=list(reversed(homebase)), tiles='cartodbpositron', zoom_start=13)

In [8]:
# Coordinates of the university:
geojson = {"type": "point", "coordinates": homebase}


pois = client.places(
    request='pois',
    geojson=geojson,
    buffer= distance_input*1000,  # searches within 500 meters of specified point [m]
    # filter_category_ids=[237,240],#[220, 568],  # https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml
    filter_category_group_ids= [220],
    validate=False,
    sortby='distance'
    # dry_run=True
)

## Output POIs

In [60]:
print("We have found the following destinations:")


for i, poi in enumerate(pois['features']):
    poi_name = poi['properties'].get('osm_tags', {}).get('name', "Kein Name zugeordnet.")
    poi_distance = poi['properties']['distance']
    poi_category_group = poi['properties'].get('category_ids',{}).values().__iter__().__next__()['category_group']

    print(f"{i+1}. : {poi_name}, Distance: {round(poi_distance/1000,2)}km")

    # depending on the category of the POI, change the icon of the marker. examples:
    # list of all icons: https://getbootstrap.com/docs/3.3/components/ (but more with the prefix 'fa' on here: https://fontawesome.com/icons?d=gallery, only works with fa-4!)
    if poi_category_group == 'historic':
        icon = folium.Icon(color="red",icon="bank", prefix='fa')#folium.Icon(prefix='fa', icon='landmark')
    elif poi_category_group == 'natural':
        icon = folium.Icon(color="green",icon="leaf", prefix='fa')#folium.Icon(prefix='fa', icon='landmark')
    else:
        icon = folium.Icon(color='blue')

    folium.Marker(
        location=list(reversed(poi['geometry']['coordinates'])),
        icon = icon,
        popup=folium.Popup(poi_name),
        tooltip=i+1
    ).add_to(m)
m

We have found the following destinations:
1. : Kein Name zugeordnet., Distance: 0.17km
2. : Gedenksäule für das Reserve-Infanterie-Regiment 273, Distance: 0.37km
3. : Kein Name zugeordnet., Distance: 0.37km
4. : Gauß-Denkmal, Distance: 0.45km
5. : Kein Name zugeordnet., Distance: 0.64km
6. : Heinrichsbrunnen, Distance: 0.75km
7. : Franz-Abt-Denkmal, Distance: 0.8km
8. : Kein Name zugeordnet., Distance: 0.82km
9. : Stadtmauerrest, Distance: 0.84km
10. : Heinrich Jasper, Distance: 0.89km
11. : Jakob Friedmann, Distance: 0.92km
12. : Horst Friedmann, Distance: 0.92km
13. : Cilla Friedmann, geb. Abelsky, Distance: 0.92km
14. : Eugen Schönlank, Elisabeth Schönlank, Arthur Schönlank, Gerhard Schönlank, Distance: 0.94km
15. : Walter Vasen, Distance: 0.96km
16. : Dina Vasen, Distance: 0.96km
17. : Samuel Vasen, Distance: 0.96km
18. : 2000 Jahre Christentum, Distance: 0.97km
19. : Fanny Moch, Heinrich Moch, Sophie Moch, Distance: 0.98km
20. : Burg Dankwarderode, Distance: 1.01km
21. : Braunschw

In [12]:
print("Where would you like to go? Choose one.")
# user input
chosen_poi_id = 31 #54 # convert that again to zero-based.

chosen_poi = pois['features'][chosen_poi_id - 1]


poi_name = chosen_poi['properties'].get('osm_tags', {}).get('name', "Kein Name zugeordnet.")
poi_distance = chosen_poi['properties']['distance']

poi_coordinates = chosen_poi['geometry']['coordinates']
print(f"You have chosen {poi_name}. Good choice!")

Where would you like to go? Choose one.
You have chosen Gedenkstätte Friedhof Hochstraße. Good choice!


### Construct polygon from coordinates

In [61]:
# since folio and the routing take different polygon/coordinate formats, we use polygon_coords and polygon_geom to store the polygon information
# folio: list of list containing tuples
# routing: MultiPolygon Object


# for each lat/long-list: create polygon, add to list and add coordinates to a separate list
# after that, create MultiPolygon from all Polygons

regions = [([10.5288243, 10.522855, 10.524474, 10.5398],[52.2733412, 52.265497, 52.264311, 52.2707]),
           ([10.5146, 10.522855, 10.524474, 10.5398],[52.270, 52.265497, 52.264311, 52.2707])]

polygon_coords = [] # coordinates are stored for folio
polygon_geom = [] # list of Polygon objects

for lon, lat in regions:
    polygon_coord = list(zip(lon, lat))
    polygon_geom.append(Polygon(polygon_coord))
    polygon_coord = [(y, x) for x, y in polygon_coord]  # Reverse coords for folium
    polygon_coords.append(polygon_coord)

polygons = MultiPolygon([p for p in polygon_geom])

# iterate over MultiPolygon (example)
# for poly in polygons.geoms:
#     print(poly)


### Plan a route to the destination restricted to these polygons (rainy areas)


In [56]:
coordinates = [homebase, poi_coordinates]

route = client.directions(
    coordinates=coordinates,
    profile='driving-car',#foot-walking',
    format='geojson',
    options={ 'avoid_polygons':  geometry.mapping(Polygon(polygon_coord))},#"avoid_features": ["steps"]},
    validate=False,
)

# request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
# route_detour = ors.directions(**request_params)

### Plot the new route and restriction polygons

In [57]:
m = folium.Map(location=list(reversed(homebase)), tiles='cartodbpositron')#, zoom_start=20)
m.fit_bounds([list(reversed(homebase)), list(reversed(poi_coordinates))], padding= (90,90))

# plot route
folium.features.GeoJson(data=route,
                        #name='Route without construction sites',
                        #style_function=style_function('#FF0000'),
                        overlay=True).add_to(m)

# add marker for homebase.
folium.Marker(
        location = list(reversed(homebase)),
        icon = folium.Icon(color="green")
    ).add_to(m)

# add marker for destination.
folium.Marker(
        location = list(reversed(poi_coordinates)),
        icon = folium.Icon(color="red")
    ).add_to(m)

# Add weather/ rain regions to the map:
#folium.GeoJson(polygon_geom).add_to(m)

for area in polygon_coords:
    folium.vector_layers.Polygon(locations=area,
                                     color='#ffd699',
                                     fill_color='#ffd699',
                                     fill_opacity=0.2,
                                     weight=3).add_to(m)

# TODO: remove the points/markers which were not chosen (or make them transparent o.Ä.)
# for now, just use a new map and add homebase and destination
m

## Next Steps

- improve looks of the polygon in folio map (order of coordinates?!)
- make sure the polygon is the exterior/convex hull and does not depend on order of coordinates
- think about structure to include wheather data, based on traveling time

## Ideas
- select the destination points in the map (but folium does not provide this)
- choose multiple points from the list and plan a route along them
- add more information to points (like ratings, images, background information)
- choose zoom level of the map dependent on distance
- show on the map, where the route has been changed due to the rain restrictions (would result in two API calls)